# Weather App | Development Notebook

In [1]:
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv
load_dotenv()
import openai
import asyncio
import nest_asyncio
from PIL import Image as PILImage
from IPython.display import Markdown, display, Image, HTML

os.chdir(os.path.dirname(os.getcwd()))

In [2]:
from typing import List, Dict, Any
import uuid
from urllib.parse import urlparse
import base64
import lancedb
import requests

In [45]:
from duckduckgo_search import DDGS

In [70]:
results = DDGS().news(keywords="austin, tx", timelimit="d", max_results=5)

In [71]:
results

[{'date': '2024-04-15T09:00:00+00:00',
  'title': 'Samsung pouring $45B into Austin area — one of largest deals in US history',
  'body': 'President Biden said the investment will "cement Central Texas\'s role as a state-of-the-art semiconductor ecosystem."',
  'url': 'https://www.bizjournals.com/austin/news/2024/04/15/samsung-chips-act-taylor-austin-joe-biden.html',
  'image': 'https://media.bizj.us/view/img/12655829/samsung-in-taylor-dec-2023*1200xx2710-1524-0-26.png',
  'source': 'The Business Journals'},
 {'date': '2024-04-15T21:04:07+00:00',
  'title': "Austin-based Tesla reportedly set to lay off 10% of its workforce. Here's what we know.",
  'body': "The cuts could affect thousands of workers if a full 10% are cut, though it's unclear how many in Texas would be affected.",
  'url': 'https://www.msn.com/en-us/money/companies/austin-based-tesla-reportedly-set-to-lay-off-10-of-its-workforce-heres-what-we-know/ar-BB1lFDwt',
  'image': 'https://www.statesman.com/gcdn/authoring/author

In [79]:
import requests
import datetime
from pprint import pprint

def celsius_to_fahrenheit(temp_celsius: float) -> float:
    return (temp_celsius * 9/5) + 32
 
API_key = os.getenv("OPENWEATHERMAP_API_KEY")
base_url = "http://api.openweathermap.org/data/2.5/weather?"
 
city_name = input("Enter a city Name : ")
final_url = base_url + "appid=" + API_key + "&q=" + city_name
 
owm_response_json = requests.get(final_url).json()
sunset_utc = datetime.datetime.fromtimestamp(owm_response_json["sys"]["sunset"])

temp_celsius = owm_response_json["main"]["temp"] - 273.15
temp_max_celsius = owm_response_json["main"]["temp_max"] - 273.15
temp_min_celsius = owm_response_json["main"]["temp_min"] - 273.15
temp_feels_like_celsius = owm_response_json["main"]["temp_min"] - 273.15

temp_fahrenheit = celsius_to_fahrenheit(temp_celsius)
temp_max_fahrenheit = celsius_to_fahrenheit(temp_max_celsius)
temp_min_fahrenheit = celsius_to_fahrenheit(temp_min_celsius)
temp_feels_like_fahrenheit = celsius_to_fahrenheit(temp_feels_like_celsius)

owm_dict = {
      "temp": owm_response_json["main"]["temp"],
      "description": owm_response_json["weather"][0]["description"],
      "icon": owm_response_json["weather"][0]["icon"],
      "sunset_utc": sunset_utc,
      "sunset_local": sunset_utc + datetime.timedelta(seconds=owm_response_json["timezone"])
  }

(
    f"In {owm_response_json["name"]}, the current weather is:\n"
    f"Detailed status: {owm_response_json["weather"][0]["description"]}\n"
    f"Wind speed: {owm_response_json["wind"]["speed"]} m/s, direction: {owm_response_json["wind"]["deg"]}°\n"
    f"Humidity: {owm_response_json["main"]["humidity"]}%\n"
    f"Temperature: \n"
    f"  - Current: {temp_fahrenheit}°F\n"
    f"  - High: {temp_max_fahrenheit}°F\n"
    f"  - Low: {temp_min_fahrenheit}°F\n"
    f"  - Feels like: {temp_feels_like_fahrenheit}°F\n"
    f"Rain: {rain if rain else 'No rain'}\n"
    f"Cloud cover: {owm_response_json["clouds"]["all"]}%"
)


print("\nCurrent Weather Data Of " + city_name +":\n")
pprint(weather_data)


Current Weather Data Of austin:

{'base': 'stations',
 'clouds': {'all': 75},
 'cod': 200,
 'coord': {'lat': 30.2672, 'lon': -97.7431},
 'dt': 1713220368,
 'id': 4671654,
 'main': {'feels_like': 303.45,
          'humidity': 73,
          'pressure': 1015,
          'temp': 300.77,
          'temp_max': 301.99,
          'temp_min': 299.58},
 'name': 'Austin',
 'sys': {'country': 'US',
         'id': 2003218,
         'sunrise': 1713182582,
         'sunset': 1713229094,
         'type': 2},
 'timezone': -18000,
 'visibility': 10000,
 'weather': [{'description': 'broken clouds',
              'icon': '04d',
              'id': 803,
              'main': 'Clouds'}],
 'wind': {'deg': 140, 'gust': 12.86, 'speed': 8.23}}


In [55]:
from pydantic import BaseModel, model_validator, field_validator
from typing import Optional, List
from datetime import datetime, timedelta
import requests
import os

class WeatherData(BaseModel):
    temp: float
    temp_max: float
    temp_min: float
    feels_like: float
    description: str
    icon: str
    wind_speed: float
    wind_direction: int
    humidity: int
    rain: str
    cloud_cover: int
    sunset_local: str
    city_name: str
    date_stamp: str

    def __str__(self):
        return (
            f"{self.date_stamp}\n"
            f"In {self.city_name}, the weather is currently:\n"
            f"Status: {self.description.title()}\n"
            f"Wind speed: {self.wind_speed} m/s, direction: {self.wind_direction}°\n"
            f"Humidity: {self.humidity}%\n"
            f"Temperature: \n"
            f"  - Current: {self.temp}°F\n"
            f"  - High: {self.temp_max}°F\n"
            f"  - Low: {self.temp_min}°F\n"
            f"  - Feels like: {self.feels_like}°F\n"
            f"Rain: {self.rain if self.rain else 'No rain'}\n"
            f"Cloud cover: {self.cloud_cover}%"
        )
    
    @property
    def to_markdown(self):
        return (
            f"{self.date_stamp}\n\n"
            f"The weather in **{self.city_name}** is currently:\n\n"
            f"Status: {self.description.title()}\n\n"
            f"Wind speed: {self.wind_speed} m/s, direction: {self.wind_direction}°\n\n"
            f"Humidity: {self.humidity}%\n\n"
            f"Temperature: \n\n"
            f"  - Current: {self.temp}°F\n"
            f"  - High: {self.temp_max}°F\n"
            f"  - Low: {self.temp_min}°F\n"
            f"  - Feels like: {self.feels_like}°F\n\n"
            f"Rain: {self.rain if self.rain else 'No rain'}\n\n"
            f"Cloud cover: {self.cloud_cover}%"
        )

def fetch_weather(search_query: str, search_type: str = "city") -> WeatherData:
    API_key = os.getenv("OPENWEATHERMAP_API_KEY")
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    
    if search_type == "city":
        final_url = f"{base_url}appid={API_key}&q={search_query}"
    elif search_type == "zip":
        final_url = f"{base_url}appid={API_key}&zip={search_query}"
    else:
        raise ValueError(f"Invalid search type: {search_type}. Must be either 'city' or 'zip'.")
    
    owm_response_json = requests.get(final_url).json()
    
    sunset_utc = datetime.fromtimestamp(owm_response_json["sys"]["sunset"])
    sunset_local = sunset_utc.strftime("%I:%M %p")
    
    temp_celsius = owm_response_json["main"]["temp"] - 273.15
    temp_max_celsius = owm_response_json["main"]["temp_max"] - 273.15
    temp_min_celsius = owm_response_json["main"]["temp_min"] - 273.15
    temp_feels_like_celsius = owm_response_json["main"]["feels_like"] - 273.15

    temp_fahrenheit = round(celsius_to_fahrenheit(temp_celsius), 2)
    temp_max_fahrenheit = round(celsius_to_fahrenheit(temp_max_celsius), 2)
    temp_min_fahrenheit = round(celsius_to_fahrenheit(temp_min_celsius), 2)
    temp_feels_like_fahrenheit = round(celsius_to_fahrenheit(temp_feels_like_celsius), 2)

    rain = owm_response_json.get("rain", "No rain")
    
    owm_dict = {
        "temp": temp_fahrenheit,
        "temp_max": temp_max_fahrenheit,
        "temp_min": temp_min_fahrenheit,
        "feels_like": temp_feels_like_fahrenheit,
        "description": owm_response_json["weather"][0]["description"],
        "icon": owm_response_json["weather"][0]["icon"],
        "wind_speed": owm_response_json["wind"]["speed"],
        "wind_direction": owm_response_json["wind"]["deg"],
        "humidity": owm_response_json["main"]["humidity"],
        "rain": rain,
        "cloud_cover": owm_response_json["clouds"]["all"],
        "sunset_local": sunset_local,
        "city_name": owm_response_json["name"],
        "date_stamp": datetime.utcnow().strftime("%A, %B %d, %Y")
    }
    
    return WeatherData(**owm_dict)

In [58]:
city = "marietta"
weather = fetch_weather('90210', search_type='zip')
print(weather)

Tuesday, April 16, 2024
In Beverly Hills, the weather is currently:
Status: Clear Sky
Wind speed: 5.66 m/s, direction: 270°
Humidity: 54%
Temperature: 
  - Current: 65.44°F
  - High: 74.3°F
  - Low: 58.06°F
  - Feels like: 64.22°F
Rain: No rain
Cloud cover: 0%
